In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact
import tkinter
import tkinter.filedialog
import tkinter.messagebox
%matplotlib inline

In [2]:
def kuwahara(pic,r=5,resize=False,rate=0.5):
    h,w,_=pic.shape
    if resize:pic=cv2.resize(pic,(int(w*rate),int(h*rate)));h,w,_=pic.shape
    pic=np.pad(pic,((r,r),(r,r),(0,0)),"edge")
    ave,var=cv2.integral2(pic)
    ave=((ave[:-r-1,:-r-1]+ave[r+1:,r+1:]-ave[r+1:,:-r-1]-ave[:-r-1,r+1:])/(r+1)**2)
    var=((var[:-r-1,:-r-1]+var[r+1:,r+1:]-var[r+1:,:-r-1]-var[:-r-1,r+1:])/(r+1)**2-ave**2).sum(axis=2)
    def filt(i,j):
        return np.array([ave[i,j],ave[i+r,j],ave[i,j+r],ave[i+r,j+r]])[(np.array([var[i,j],var[i+r,j],var[i,j+r],var[i+r,j+r]]).argmin(axis=0).flatten(),j.flatten(),i.flatten())].reshape(w,h,_).transpose(1,0,2)
    filtered_pic = filt(*np.meshgrid(np.arange(h),np.arange(w))).astype(pic.dtype)
    return filtered_pic

In [3]:
tkinter.Tk().withdraw()
pic=np.array(plt.imread(tkinter.filedialog.askopenfilename()))

@interact(r=(1,30),rate=(0,100),T=(0,1))
def interactkuwa(r,rate,T):
    im=pic.copy()
    plt.figure(figsize=(15, 15))
    filtered_pic=kuwahara(im,r,1,rate/100)
    if T:plt.imshow(filtered_pic.transpose(1,0,2)[:,::-1])
    else:plt.imshow(filtered_pic)

interactive(children=(IntSlider(value=15, description='r', max=30, min=1), IntSlider(value=50, description='ra…